# Retrieval Augmented Generation

We will use the semantic search to provide the best matching wine based on the review description. [Retrieval Augmented Generation](https://arxiv.org/abs/2005.11401) is a process that combines retrieval-based models and generative models to enhance natural language generation by retrieving relevant information and incorporating it into the generation process. In this notebook, we'll walk through enhancing an OpenSearch cluster search with generative AI to output conversational wine recommendations based on a desired description.

### 1. Install OpenSearch ML Python library

For this notebook we require the use of a few key libraries. We'll use the Python clients for OpenSearch and SageMaker, and Python frameworks for text embeddings.

In [ ]:
!pip install opensearch-py-ml accelerate sentence-transformers tqdm --quiet
!pip install sagemaker --upgrade --quiet

### 2. Check PyTorch Version


As in the previous modules, let's import PyTorch and confirm that the latest version of PyTorch is running. The version should already be 1.13.1 or higher. If not, please run the lab in order to get everything set up.

In [ ]:
import torch
print(torch.__version__)

### 3. Retrieve notebook variables

The line below will retrieve your shared variables from the previous notebook.

In [ ]:
%store -r

Now we need to restart the kernel by running below cell.

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

### 4. Import libraries
The line below will import all the relevant libraries and modules used in this notebook.

In [ ]:
import boto3
import os
import time
import json
import pandas as pd
from tqdm import tqdm
import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection
from transformers import AutoTokenizer, AutoModel, DistilBertTokenizer, DistilBertModel
from sagemaker import get_execution_role, image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
print(sagemaker.__version__)

### 5. Prepare data

This lab combines semantic search with a generative model to present the retrieved data to the user in a conversational tone. Below is a dataset of wine reviews, we'll sample this data set to recommend wines that resemble the user provided description.

### Note
You can download the dataset from various sources. One is Kaggle.
https://www.kaggle.com/datasets/christopheiv/winemagdata130k?select=winemag-data-130k-v2.json

After downloading and copying here, unzip in the working directory if it hasn't already been unzipped. Execute the following cells to inspect the dataset, transform it into a pandas DataFrame, and sample a subset of the data.

In [ ]:
#!unzip -o winemag-data-130k-v2.json.zip

In [ ]:
df = pd.read_json('winemag-data-130k-v2.json')

df.sample(3)

In [ ]:
df.columns

In [ ]:
wm_list = df.sample(300,
                   random_state=37).to_dict('records') # sample to keep lab quick

wm_list[:1]

### 6. Create an OpenSearch cluster connection.
Next, we'll use Python API to set up connection with OpenSearch Cluster.

Note: if you're using a region other than us-east-1, please update the region in the code below.

#### Get Cloud Formation stack output variables

We also need to grab some key values from the infrastructure we provisioned using CloudFormation. To do this, we will list the outputs from the stack and store this in "outputs" to be used later.

You can ignore any "PythonDeprecationWarning" warnings.

In [ ]:
region = 'us-east-1' 

cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "semantic-search"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']

outputs

In [ ]:
auth = ("master","Semantic123!")
aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

### 7. Download and Deploy Text Embeddings Endpoint

Similar to the previous modules, we will be using the BERT model to generate vectorization data, where every sentence is 768 dimension data. As shown in Modules 3 and 4, embedding text allows us to search the cluster more accurately than the default text matching. 

We will be downloading the pre-trained embeddings model and deploying it as a SageMaker endpoint for inference. Deploying the model may take a few minutes.

In [ ]:
model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
saved_model_dir = 'transformer'
os.makedirs(saved_model_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name) 

tokenizer.save_pretrained(saved_model_dir)
model.save_pretrained(saved_model_dir)

Create a compressed model file `model.tar.gz` in the parent directory from the downloaded model so that we can deploy the model to an endpoint.

In [ ]:
!cd transformer && tar czvf ../model.tar.gz *

Upload `model.tar.gz` and deploy an inference endpoint to embed text. This cell may take a few minutes to execute.

In [ ]:
class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='sentence-transformers-model')

pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code',
                             py_version = 'py39', 
                             framework_version = '1.13.1',
                             predictor_cls=StringPredictor)



embed_predictor = pytorch_model.deploy(instance_type='ml.m5d.large', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'distilbert-embedding-model-{int(time.time())}')

If you already deployed a model, skip the execution of the previous cell, uncomment the below cell, and add your endpoint name to `embed_endpoint_name`.

In [ ]:
# embed_endpoint_name = "endpoint_name"
# sagemaker_session = sagemaker.Session()
# embed_predictor = StringPredictor(endpoint_name=embed_endpoint_name, 
#                                   sagemaker_session = sagemaker_session)

### 8. Test the embeddings endpoint with a sample phrase
Using any text phrase, the endpoint converts the text to a vectorized array of size 768. We're also creating a function `embed_phrase` so that we can call it later.

In [ ]:
def embed_phrase(phrase):
    features = embed_predictor.predict(phrase)
    return json.loads(features)

embed_phrase("pairs well with chocolate")

### 9. Create a index in Amazon Opensearch Service 
Whereas we previously created an index with 2-3 fields, this time we'll define the index with multiple fields: the vectorization of the `description` field, and all others present within the dataset.

To create the index, we first define the index in JSON, then use the aos_client connection we initiated ealier to create the index in OpenSearch.

In [ ]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "description_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "store": True
            },
            "description": {
                "type": "text",
                "store": True
            },
            "designation": {
                "type": "text",
                "store": True
            },
            "variety": {
                "type": "text",
                "store": True
            },
            "country": {
                "type": "text",
                "store": True
            },
            "winery": {
                "type": "text",
                "store": True
            },
            "points": {
                "type": "integer",
                "store": True
            },
        }
    }
}


In [ ]:
# aos_client.indices.delete(index="nlp_pqa") # drop the index from the previous lab
# If this is the first time you're running this, you won't have this index to drop

Using the above index definition, we now need to create the index in Amazon OpenSearch. Running this cell will recreate the index if you have already executed this notebook.

In [ ]:
index_name = "nlp_wmd"

try:
    aos_client.indices.delete(index=index_name)
    print("Recreating index '" + index_name + "' on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)
except:
    print("Index '" + index_name + "' not found. Creating index on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)


Let's verify the created index information

In [ ]:
aos_client.indices.get(index="nlp_wmd")

### 10. Load the raw data into the Index
Next, let's load the wine review data into the index we've just created. During ingest data defined by the `os_import` function, `description` field will also be converted to vector (embedding) by calling the previously created endpoint.

In [ ]:
def os_import(record, aos_client, index_name):
    description = record["description"]
    search_vector = embed_phrase(description)
    aos_client.index(index=index_name,
             body={"description_vector": search_vector, 
                   "description": record["description"],
                   "points":record["points"],
                   "variety":record["variety"],
                   "country":record["country"],
                   "designation":record["designation"],
                   "winery":record["winery"]
                  }
            )

print("Index created. Loading records...")
for record in tqdm(wm_list): 
    os_import(record, aos_client, index_name)
print("Records loaded.")

To validate the load, we'll query the number of documents number in the index. We should have 300 hits in the index, or however many was specified earlier in sampling.

In [ ]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d." % res['hits']['total']['value'])

### 11. Search vector with "Semantic Search" 

Now we can define a helper function to execute the search query for us to find a wine whose review most closely matches the requested description. `query_opensearch` embeds the search phrase, searches the cluster index for the closest matching vector, and returns the top result.


In [ ]:
def query_opensearch(phrase, n=1):
    search_vector = embed_phrase(phrase)
    osquery={
        "_source": {
            "exclude": [ "description_vector" ]
        },
        
      "size": n,
      "query": {
        "knn": {
          "description_vector": {
            "vector":search_vector,
            "k":n
          }
        }
      }
    }

    res = aos_client.search(index="nlp_wmd", 
                           body=osquery,
                           stored_fields=["description","winery","points", "designation", "country"],
                           explain = True)
    top_result = res['hits']['hits'][0]
    
    result = {
        "description":top_result['_source']['description'],
        "winery":top_result['_source']['winery'],
        "points":top_result['_source']['points'],
        "designation":top_result['_source']['designation'],
        "country":top_result['_source']['country'],
        "variety":top_result['_source']['variety'],
    }
    
    return result

example_request = query_opensearch("A wine that pairs well with meat.")
print(example_request)

### 12. Deploy the Large Language Model for Retrieval Augmented Generation

This module uses the [Falcon 7B](https://falconllm.tii.ae) model to create recommendations based on a given wine review. The next cell deploys a model endpoint into your environment that will be called by subsequent steps. For more information on the Falcon LLM, see [HuggingFace's announcement](https://huggingface.co/blog/falcon) regarding the model.

In [ ]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)

model_name = "falcon-7b-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'tiiuae/falcon-7b',
    'HF_TASK':'question-answering',
    'SM_NUM_GPUS':'1',
    'HF_MODEL_QUANTIZE':'bitsandbytes'
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)

instance_type = "ml.g5.4xlarge"
#Other supported 1-GPU instance types: ml.g5.2xlarge, ml.g5.xlarge
#With instances with more GPUs, change the SM_NUM_GPUS value within the hub variable
#For example, using endpoint instance ml.g5.12xlarge, specify 'SM_NUM_GPUS':'4'

print("☕ Spinning up the endpoint. This will take a little while ☕")

llm_predictor = model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  endpoint_name=model_name
)

In [ ]:
# If you already deployed a model, 
# uncomment the following lines and add your endpoint name below

# from sagemaker.huggingface import HuggingFacePredictor
# sagemaker_session = sagemaker.Session()
# llm_endpoint_name = "endpoint_name"
# llm_predictor = HuggingFacePredictor(endpoint_name=llm_endpoint_name, sagemaker_session = sagemaker_session)

### 13. Create a prompt for the LLM using the search results from OpenSearch

We will be using the Falcon-7B model for single-shot generation, using a canned recommendation and response to guide the output. Before querying the model, the below function `render_prompt` is used to easily make a prompt for single-shot generation. The function takes in an input string to search the OpenSearch cluster for a matching wine, and outputs a viable prompt.

In [ ]:
def render_prompt(requested_description):
    recommendation = query_opensearch(requested_description)
    sample_recommendation = "{{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}}"
    sample_response = "I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator."
    prompt = (
        f"[CLM] Context: A sommelier uses their vast knowledge of wine to make great recommendations people will enjoy. A recommendation always includes the winery, the country of origin, and a colorful description."
        f"Data: <br> Recommendation: <br>"
        f"Data: {sample_recommendation} <br> Recommendation: {sample_response} <br>"
        f"Data: {recommendation} <br> Recommendation:"
    )
    
    return prompt

prompt = render_prompt("pairs well with meat") 
print(prompt)

### 14. Format LLM input and query using the rendered prompt
We also need a few more helper functions to query the LLM. `render_llm_input` transforms the generated prompt into the correct input format, `render_llm_output` parses the LLM output. 

`query_llm` combines everything we've done in this module. It does all of the following:
- embeds the input (the desired description of a wine)
- searches the OpenSearch index for the closest description vector
- renders an LLM prompt from the search results
- queries the LLM for a conversational response

In [ ]:
def render_llm_input(data, **kwargs):
    default_kwargs = {
        "num_beams": 5,
        "no_repeat_ngram_size": 3,
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.6,
        "watermark": True,
        "top_k": 147,
        "max_length": 175,
        "early_stopping": True
    }
    
    default_kwargs = {**default_kwargs, **kwargs}
    
    input_data = {
        "inputs": data,
        "parameters": default_kwargs
    }
    
    return input_data

def render_llm_output(response):
    return response.split("Recommendation: ")[-1].split("<br>")[0].split("Data:")[0].split("\n")[0]

def query_llm(description, **kwargs):
    prompt = render_prompt(description)
    query = render_llm_input(prompt, **kwargs)
    response = llm_predictor.predict(query)[0]['generated_text']
    rec = render_llm_output(response)
    return rec

#### And finally, let's call the function and get a wine recommendation.

In [ ]:
recommendation = query_llm("A wine that pairs well with meat.")
print(recommendation)

### Additional info: changing kwargs for querying the LLM
If you want to change or add new parameters for LLM querying, you're able to add in new keyword arguments to the `query_llm` function. For example, to change the `temperature` value, simply change the function call:
`query_llm(description phrase, temperature = new float value)`

## Cleaning Up
After you've finished using the endpoints, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
embed_predictor.delete_endpoint()
llm_predictor.delete_endpoint()